#JPEG Compression

###Import Modules

In [20]:
# Import the required modules
import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
import os

###Quantization Matrix

In [21]:
# Define the quantization matrix
quantization_matrix = np.array(
    [
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99],
    ],
    dtype=np.float32,
)
quantization_matrix_Y = np.array(
    [
        [16, 11, 10, 16, 24, 40, 51, 61],
        [12, 12, 14, 19, 26, 58, 60, 55],
        [14, 13, 16, 24, 40, 57, 69, 56],
        [14, 17, 22, 29, 51, 87, 80, 62],
        [18, 22, 37, 56, 68, 109, 103, 77],
        [24, 35, 55, 64, 81, 104, 113, 92],
        [49, 64, 78, 87, 103, 121, 120, 101],
        [72, 92, 95, 98, 112, 100, 103, 99],
    ],
    dtype=np.float32,
)

quantization_matrix_CrCb = np.array(
    [
        [17, 18, 24, 47, 99, 99, 99, 99],
        [18, 21, 26, 66, 99, 99, 99, 99],
        [24, 26, 56, 99, 99, 99, 99, 99],
        [47, 66, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
        [99, 99, 99, 99, 99, 99, 99, 99],
    ],
    dtype=np.float32,
)

###PSNR

The calculate_psnr function computes the Peak Signal-to-Noise Ratio (PSNR) between two images, img1 and img2, which is a common measure of image quality, especially in compression. It calculates the Mean Squared Error (MSE) by taking the average of the squared differences between corresponding pixels in the two images. Then, it uses the formula for PSNR, where a higher PSNR value indicates better quality (closer match between images). Here, 255 represents the maximum possible pixel intensity for an 8-bit grayscale image.

In [22]:
def calculate_psnr(img1: np.ndarray, img2: np.ndarray) -> float:
    """Calculate PSNR using formula: PSNR = 20 * log10(MAX_I) - 10 * log10(MSE)"""
    mse = np.mean((img1 - img2) ** 2)  # Calculate the Mean Squared Error (MSE) between the two images
    psnr = 20 * np.log10(255 / np.sqrt(mse))  # Compute PSNR using the maximum pixel intensity (255 for 8-bit images) and MSE
    return psnr  # Return the calculated PSNR value

###Number of Elements

The number_of_elements function calculates the total number of significant (non-zero) elements in a list of 1D arrays representing blocks of a grayscale JPEG-encoded image. It iterates through each block, trims trailing zeros (commonly produced during JPEG compression), and adds the count of elements in each trimmed block to get the final count. This helps capture the actual data stored without excess zeros.

The total_number_of_elements function provides a higher-level count of non-zero elements for either grayscale or color images. For color images, it separately counts elements in each channel (e.g., R, G, B) using number_of_elements for each channel's blocks, then sums them. For grayscale images, it simply calls number_of_elements on the entire block list. This function is particularly useful for calculating compression ratios by measuring the effective data size across different image formats.

In [23]:
def number_of_elements(blocks: list[np.ndarray]) -> int:
    """Calculates the total number of elements in the grayscale JPEG encoded array"""
    total_elements = 0  # Initialize the total element count to zero
    for block in blocks:  # Iterate through each block in the list
        # Trim the trailing zeros from the 1D array (helps reduce unnecessary zeros in compression)
        total_elements += np.trim_zeros(block, "b").size  # Add the size of the trimmed array to the total
    return total_elements  # Return the final count of total elements

def total_number_of_elements(
    blocks: list[np.ndarray]
    | tuple[
        list[np.ndarray],
        list[np.ndarray],
        list[np.ndarray],
    ],
    color: bool,
) -> int:
    """
    Calculates the total number of elements for both color and grayscale JPEG encoded arrays
    This is a utility function that will be used to calculate the compression ratio
    """
    total_elements = 0  # Initialize the total element count to zero
    if color:  # If the image is in color (has multiple channels)
        # Sum the number of elements across each color channel (e.g., R, G, B)
        total_elements = (
            number_of_elements(blocks[0])  # Count elements in the first channel
            + number_of_elements(blocks[1])  # Count elements in the second channel
            + number_of_elements(blocks[2])  # Count elements in the third channel
        )
    else:  # If the image is grayscale (single channel)
        total_elements = number_of_elements(blocks)  # Count elements in the grayscale channel
    return total_elements  # Return the final count of total elements

##ZigZag Scan, Unscan

###ZigZag Scan

The zigzag_scan function rearranges the elements of a 2D square matrix (block) into a 1D array following a zigzag pattern, commonly used in image compression techniques like JPEG. It does this by extracting the diagonals of the matrix and reversing the order for every alternate diagonal to create the zigzag effect. The result is a 1D array with elements ordered from top-left to bottom-right, optimizing data for subsequent compression.

In [24]:
def zigzag_scan(block: np.ndarray) -> np.ndarray:
    """
    Scans a block in zigzag order and returns a 1D array.
    Each block is assumed to be a square matrix.
    """
    a = 0  # Placeholder variable, likely unused (can be removed if not used elsewhere)
    block_size = block.shape[0]  # Determine the size of the square block (assumes block is square)

    # Generate a 1D array by scanning diagonals of the block in a zigzag pattern
    zigzag_arr = np.concatenate(
        [
            np.diagonal(block[::-1, :], i)[:: (2 * (i % 2) - 1)]  # Extract diagonals in zigzag order
            for i in range(1 - block_size, block_size)  # Iterate over possible diagonals in the block
        ]
    )
    return zigzag_arr  # Return the 1D array of elements in zigzag order

###ZigZag Unscan

The zigzag_unscan function reconstructs a 2D square matrix from a 1D array following a zigzag pattern, often reversing the zigzag scan used in compression processes like JPEG. Starting at the top-left, it places each element of the 1D array into the correct (x, y) position in the 2D matrix by adjusting movement direction based on the current coordinate sum (even or odd). This reordering is crucial for reconstructing compressed data into its original 2D block form.

In [25]:
def zigzag_unscan(
    zigzag_arr: np.ndarray, block_size: int
) -> np.ndarray:
    """Unscans a 1D array in zigzag order and returns a 2D array."""

    # Create an empty 2D array to store the unscanned values
    block = np.zeros((block_size, block_size), dtype=np.float32)  # Initialize a 2D array of the given block size
    x, y = 0, 0  # Initialize coordinates at the top-left corner

    for num in zigzag_arr:  # Iterate through each element in the 1D zigzag array
        block[x, y] = num  # Place the current element in the (x, y) position of the 2D array

        # Determine the direction to move based on the current position
        # If the sum of the coordinates is even, move in the "up-right" zigzag pattern
        if (x + y) % 2 == 0:
            # If at the last column, move down one row
            if y == block_size - 1:
                x += 1
            # If at the first row, move right one column
            elif x == 0:
                y += 1
            # Otherwise, move up one row and right one column
            else:
                x -= 1
                y += 1
        # If the sum of the coordinates is odd, move in the "down-left" zigzag pattern
        else:
            # If at the last row, move right one column
            if x == block_size - 1:
                y += 1
            # If at the first column, move down one row
            elif y == 0:
                x += 1
            # Otherwise, move down one row and left one column
            else:
                x += 1
                y -= 1

    return block  # Return the reconstructed 2D array

##Grayscale JPEG Encoder, Decoder

###Grayscale JPEG Encoder

The grayscale_jpeg_encoder function compresses a grayscale image using JPEG-like encoding by processing it in blocks. It first pads the image to make the dimensions divisible by the block size, then subtracts 128 from the pixel values to shift the image to a zero-centered color space. The image is divided into non-overlapping blocks, and a Discrete Cosine Transform (DCT) is applied to each block to transform the data into frequency space. The quantization matrix is resized to match the block size, and each DCT coefficient is quantized by dividing it by the resized matrix. After quantization, the function performs zigzag scanning on each block to generate a 1D array of coefficients. Finally, the first num_coefficients coefficients are retained for each block and returned as the JPEG-encoded array.

In [26]:
def grayscale_jpeg_encoder(
    img: np.ndarray, block_size: int, num_coefficients: int, quantization_matrix
) -> list[np.ndarray]:
    """
    Encodes a grayscale image using JPEG compression.
    Returns a list of 1D arrays containing the first `num_coefficients`
    coefficients after performing zigzag scanning on each quantized block.
    This is the JPEG encoded array.
    """

    # Pad the image to make its height and width divisible by the block size
    height, width = img.shape  # Get the height and width of the input image
    padded_height = height + (block_size - height % block_size) % block_size  # Calculate padded height
    padded_width = width + (block_size - width % block_size) % block_size  # Calculate padded width
    padded_img = np.zeros((padded_height, padded_width), dtype=np.uint8)  # Initialize padded image with zeros
    padded_img[:height, :width] = img  # Copy the original image into the padded image

    # Subtract 128 from the image to center pixel values around zero (JPEG color space adjustment)
    padded_img = padded_img.astype(np.float32) - 128  # Convert to float32 and subtract 128 for DC shift

    # Split the image into non-overlapping blocks of the given block size
    blocks = [
        padded_img[i : i + block_size, j : j + block_size]
        for i in range(0, padded_height, block_size)
        for j in range(0, padded_width, block_size)
    ]  # Generate blocks of the image for DCT processing

    # Apply the Discrete Cosine Transform (DCT) to each block to transform to frequency domain
    dct_blocks = [cv.dct(block) for block in blocks]  # DCT transformation for each block

    # Resize the quantization matrix to match the block size for consistency
    resized_quantization_matrix = cv.resize(
        quantization_matrix, (block_size, block_size), cv.INTER_CUBIC
    )  # Resize quantization matrix using cubic interpolation

    # Quantize each DCT coefficient by dividing each DCT value by the resized quantization matrix
    quantized_blocks = [
        np.round(block / resized_quantization_matrix).astype(np.int32)
        for block in dct_blocks
    ]  # Quantize each block by rounding the coefficients

    # Perform zigzag scanning on each quantized block to create a 1D array
    zigzag_scanned_blocks = [zigzag_scan(block) for block in quantized_blocks]  # Apply zigzag scan

    # Retain only the first `num_coefficients` coefficients in each zigzag-scanned block
    first_num_coefficients = [
        block[:num_coefficients] for block in zigzag_scanned_blocks
    ]  # Retain the specified number of coefficients

    return first_num_coefficients  # Return the list of quantized and zigzag-scanned coefficients

###Grayscale JPEG Decoder

The grayscale_jpeg_decoder function reverses the JPEG-like compression applied to a grayscale image. It begins by padding the image dimensions to align with the block size. It then resizes the quantization matrix to match the block size and reverses the zigzag scanning applied during encoding to obtain the quantized blocks. After dequantizing each block by multiplying with the resized quantization matrix, the function applies the Inverse Discrete Cosine Transform (IDCT) to each block to revert the frequency domain data to spatial data. The IDCT blocks are placed in their respective positions to reconstruct the padded image. The DC shift introduced during encoding is corrected by adding 128 to each pixel value, and the pixel values are clipped to ensure they remain within the valid range. Finally, the image is cropped back to its original size and returned as a uint8 array, ready for display or further processing.

In [27]:
def grayscale_jpeg_decoder(
    blocks: list[np.ndarray], img: np.ndarray, block_size: int, quantization_matrix
) -> np.ndarray:
    """
    Decodes a grayscale image using JPEG compression from the JPEG encoded array.
    Returns a 2D array containing the compressed image.
    """

    # Calculate the padded height and width of the image to handle non-multiple dimensions
    height, width = img.shape  # Get the height and width of the input image
    padded_height = height + (block_size - height % block_size) % block_size  # Calculate padded height
    padded_width = width + (block_size - width % block_size) % block_size  # Calculate padded width

    # Resize the quantization matrix to match the block size for consistency
    resized_quantization_matrix = cv.resize(
        quantization_matrix, (block_size, block_size), cv.INTER_CUBIC
    )  # Resize the quantization matrix using cubic interpolation to match block size

    # Unscan the zigzag-scanned blocks to recover the quantized DCT blocks
    zigzag_unscanned_blocks = [zigzag_unscan(block, block_size) for block in blocks]  # Reverse zigzag scanning

    # Dequantize the blocks by multiplying each by the resized quantization matrix
    dequantized_blocks = [
        block * resized_quantization_matrix for block in zigzag_unscanned_blocks
    ]  # Dequantize by element-wise multiplication with the quantization matrix

    # Apply the Inverse Discrete Cosine Transform (IDCT) to each dequantized block to recover spatial domain data
    idct_blocks = [cv.idct(block) for block in dequantized_blocks]  # Perform IDCT on each block

    # Reconstruct the compressed image from the IDCT blocks by placing them back in the original positions
    compressed_img = np.zeros((padded_height, padded_width), dtype=np.float32)  # Initialize the padded image array
    block_index = 0  # Index to track the current block
    for i in range(0, padded_height, block_size):  # Loop through rows
        for j in range(0, padded_width, block_size):  # Loop through columns
            compressed_img[i : i + block_size, j : j + block_size] = idct_blocks[
                block_index
            ]  # Place the IDCT block in the corresponding position
            block_index += 1  # Increment the block index

    # Add 128 to shift pixel values back to the original range
    compressed_img += 128  # Add 128 to reverse the DC shift from encoding

    # Clip the values to ensure they stay within the valid pixel range (0-255)
    compressed_img = np.clip(compressed_img, 0, 255)  # Clip pixel values to the range [0, 255]

    # Crop the image back to its original size and convert to uint8 for image format
    return compressed_img[:height, :width].astype(np.uint8)  # Return the cropped and properly formatted image

##Color JPEG Encoder, Decoder

###Color JPEG Encoder

The color_jpeg_encoder function performs JPEG compression for a color image in the YCbCr color space. It first converts the input BGR image into the YCbCr color space, which separates the luminance (Y) component from the chrominance (Cb, Cr) components. Then, the image is split into the individual Y, Cb, and Cr channels. Each channel is processed separately using the grayscale_jpeg_encoder function to apply JPEG compression. The function returns a tuple containing three lists: the first list corresponds to the Y channel (luminance), and the other two lists correspond to the Cb and Cr channels (chrominance). These lists contain 1D arrays representing the first num_coefficients after zigzag scanning and quantization, which is the JPEG encoded data for each channel.

In [28]:
def color_jpeg_encoder(
    img: np.ndarray, block_size: int, num_coefficients: int
) -> tuple[
    list[np.ndarray], list[np.ndarray], list[np.ndarray]
]:
    """
    Encodes a color image using JPEG compression in YCbCr color space.
    Returns a tuple of 3 lists, each containing
    1D arrays containing the first `num_coefficients`
    coefficients after performing zigzag scanning on each quantized block.
    This is the JPEG encoded array.
    The three lists correspond to the Y, Cb, and Cr channels respectively.
    """

    # Convert the image to YCbCr color space, which separates luminance (Y) and chrominance (Cb, Cr)
    ycbcr_image = cv.cvtColor(img, cv.COLOR_BGR2YCrCb)  # Convert BGR image to YCbCr color space

    # Split the YCbCr image into separate Y, Cb, and Cr channels
    y_channel, cb_channel, cr_channel = cv.split(ycbcr_image)  # Split the YCbCr image into Y, Cb, Cr

    # Encode each channel using grayscale_jpeg_encoder
    # Each channel is encoded separately using the grayscale JPEG encoder function
    return (
        grayscale_jpeg_encoder(y_channel, block_size, num_coefficients, quantization_matrix_Y),  # Encode Y channel
        grayscale_jpeg_encoder(cb_channel, block_size, num_coefficients, quantization_matrix_CrCb),  # Encode Cb channel
        grayscale_jpeg_encoder(cr_channel, block_size, num_coefficients, quantization_matrix_CrCb),  # Encode Cr channel
    )

###Color JPEG Decoder

The color_jpeg_decoder function decodes a JPEG-encoded color image that has been compressed in the YCbCr color space. The process begins by converting the input BGR image to the YCbCr color space. The image is then split into three separate channels: Y (luminance), Cb (chrominance-blue), and Cr (chrominance-red). Each channel is individually decoded using the grayscale_jpeg_decoder function, which reverses the JPEG compression for each channel. After decoding, the Y, Cb, and Cr channels are merged back into a single YCbCr image. Finally, the YCbCr image is converted back to the BGR color space for visualization, and the decoded BGR image is returned.

In [29]:
def color_jpeg_decoder(
    blocks: tuple[
        list[np.ndarray],
        list[np.ndarray],
        list[np.ndarray],
    ],
    img: np.ndarray,
    block_size: int,
) -> np.ndarray:
    """
    Decodes a JPEG encoded color image in YCbCr color space.
    Returns a 3D array containing the compressed image.
    """

    # Convert the image to YCbCr color space for decoding
    ycbcr_image = cv.cvtColor(img, cv.COLOR_BGR2YCrCb)  # Convert the input BGR image to YCbCr color space

    # Split the YCbCr image into separate Y, Cb, and Cr channels
    y_channel, cb_channel, cr_channel = cv.split(ycbcr_image)  # Split into Y, Cb, and Cr channels

    # Decode each channel using the grayscale_jpeg_decoder function
    # Each channel is decoded separately by applying the inverse process of JPEG compression
    y_channel = grayscale_jpeg_decoder(blocks[0], y_channel, block_size, quantization_matrix_Y)  # Decode Y channel
    cb_channel = grayscale_jpeg_decoder(blocks[1], cb_channel, block_size, quantization_matrix_CrCb)  # Decode Cb channel
    cr_channel = grayscale_jpeg_decoder(blocks[2], cr_channel, block_size, quantization_matrix_CrCb)  # Decode Cr channel

    # Merge the decoded Y, Cb, and Cr channels back into a single YCbCr image
    ycbcr_decoded = cv.merge((y_channel, cb_channel, cr_channel))  # Merge the decoded channels into YCbCr image

    # Convert the YCbCr image back to BGR color space for display
    bgr_decoded = cv.cvtColor(ycbcr_decoded, cv.COLOR_YCrCb2RGB)  # Convert YCbCr to BGR for standard display

    return bgr_decoded

##JPEG Encoder, Decoder

###JPEG Encoder

The jpeg_encoder function handles JPEG compression for both color and grayscale images. It takes the image file path, block size, the number of coefficients to retain, and a flag indicating whether the image is in color. Depending on the color flag, the function either loads a color image or a grayscale image using OpenCV’s imread function. If the image is color, the function applies the color_jpeg_encoder, which processes the image in the YCbCr color space and returns the JPEG-encoded data. If the image is grayscale, it applies the grayscale_jpeg_encoder to compress the image and return the encoded data. The function returns a list for grayscale images or a tuple for color images containing the JPEG encoded data for each channel.

In [30]:
def jpeg_encoder(
    img_path: str,
    block_size: int,
    num_coefficients: int,
    color: bool,
) -> (
    list[np.ndarray]
    | tuple[
        list[np.ndarray],
        list[np.ndarray],
        list[np.ndarray],
    ]
):
    """
    Encodes an image using JPEG compression.
    Returns the JPEG encoded array.
    """

    if color:  # Check if the image is color or grayscale
        # Load the color image from the specified path
        img = cv.imread(img_path, cv.IMREAD_COLOR)  # Read the image as a color (BGR) image

        # Apply color JPEG encoding by passing the image to the color JPEG encoder
        return color_jpeg_encoder(img, block_size, num_coefficients)  # Return the JPEG encoded color image

    else:
        # Load the grayscale image from the specified path
        img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)  # Read the image as a grayscale image

        # Apply grayscale JPEG encoding by passing the image to the grayscale JPEG encoder
        return grayscale_jpeg_encoder(img, block_size, num_coefficients, quantization_matrix)  # Return the JPEG encoded grayscale image

###JPEG Decoder

The jpeg_decoder function decodes a JPEG compressed image from its encoded array, depending on whether it is a color or grayscale image. It takes the encoded blocks, the image path, block size, and a flag (color) that determines if the image is in color or grayscale.

1. If color is True, the function loads the color image using OpenCV and applies the color_jpeg_decoder to decode the JPEG image, returning a 3D array of the decoded color image.
2. If color is False, the function loads the grayscale image and applies the grayscale_jpeg_decoder, returning a 2D array of the decoded grayscale image.

This function returns either a 2D or 3D array representing the decompressed image based on the image type.

In [31]:
def jpeg_decoder(
    blocks: list[np.ndarray]
    | tuple[
        list[np.ndarray],
        list[np.ndarray],
        list[np.ndarray],
    ],
    img_path: str,
    block_size: int,
    color: bool,
) -> np.ndarray:
    """
    Decodes an image using JPEG compression from its JPEG encoded array.
    Returns a 2D or 3D array containing the compressed image.
    """

    if color:  # Check if the image is color or grayscale
        # Load the color image from the specified path
        img = cv.imread(img_path, cv.IMREAD_COLOR)  # Read the image as a color (BGR) image

        # Apply color JPEG decoding using the provided blocks and return the decoded image
        return color_jpeg_decoder(blocks, img, block_size)  # Decode the color JPEG image

    else:
        # Load the grayscale image from the specified path
        img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)  # Read the image as a grayscale image

        # Apply grayscale JPEG decoding using the provided blocks and return the decoded image
        return grayscale_jpeg_decoder(blocks, img, block_size, quantization_matrix)  # Decode the grayscale JPEG image

##Analyze image

The analyze_image function performs JPEG compression on an image, analyzes its quality, and returns multiple metrics for comparison. It follows these steps:

1. **Reads the image**: The image is read from the specified path based on whether it’s in color or grayscale.
2. **JPEG encoding**: The image is compressed using JPEG encoding, and the result is stored in encoded_img.
3. **JPEG decoding**: The encoded image is then decoded back to get the compressed version.
4. **PSNR Calculation**: The Peak Signal-to-Noise Ratio (PSNR) is computed to evaluate the quality of the compressed image.
5. **Compression Ratio Calculation**: The compression ratio is calculated as the ratio of the original image size to the encoded image size.
6. **Returns multiple outputs**:
  * The original image
  * The compressed image
  * The PSNR value (indicating image quality loss)
  * The compression ratio
  * The encoded image (for inspection or further processing)
  * A flag (color) indicating whether the image is in color or grayscale.

This function helps in comparing the quality and efficiency of the JPEG compression algorithm.

In [32]:
def analyze_image(
    img_path: str, block_size: int, num_coefficients: int, color: bool
) -> tuple[
    np.ndarray,
    np.ndarray,
    float,
    float,
    list[np.ndarray]
    | tuple[
        list[np.ndarray],
        list[np.ndarray],
        list[np.ndarray],
    ],
    bool,
]:
    """
    Analyzes the input image by performing JPEG compression,
    Returns the original and compressed images, and the PSNR and compression ratio.
    This can be used to compare the quality of the compressed image.
    """

    # Read the image based on the color flag
    img: np.ndarray = None  # Initialize the image variable
    if color:  # If the image is in color
        img = cv.imread(img_path, cv.IMREAD_COLOR)  # Read the color image
    else:  # If the image is grayscale
        img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)  # Read the grayscale image

    # Encode the image using JPEG compression
    encoded_img = jpeg_encoder(img_path, block_size, num_coefficients, color)  # Perform JPEG encoding

    # Decode the image using JPEG compression
    compressed_img = jpeg_decoder(encoded_img, img_path, block_size, color)  # Perform JPEG decoding

    # Calculate the Peak Signal-to-Noise Ratio (PSNR) between the original and compressed images
    psnr = cv.PSNR(img, compressed_img)  # PSNR is a metric to measure image quality loss

    # Calculate the compression ratio
    n2 = total_number_of_elements(encoded_img, color)  # Get the total number of elements in the encoded image
    if n2 == 0:  # If the total number of elements is 0, avoid division by zero
        compression_ratio = 0  # Set compression ratio to 0
    else:
        compression_ratio = img.size / total_number_of_elements(encoded_img, color)  # Compute compression ratio

    # Return a tuple containing:
    # - the original image,
    # - the compressed image,
    # - PSNR value,
    # - compression ratio,
    # - the encoded image (for possible saving or inspection),
    # - a flag indicating whether the image is color or grayscale
    return (img, compressed_img, psnr, compression_ratio, encoded_img, color)

##Save Compressed Image

In [33]:
def save_compressed_image(
    compressed_img: np.ndarray,
    compression_ratio: float,
) -> None:
    # Generate the filename for the compressed image, using the compression ratio
    compressed_img_filename = f"compressed_{compression_ratio:.2f}.jpeg"

    # Display the compressed image using matplotlib
    plt.imshow(compressed_img, cmap="gray")  # Assuming grayscale image, use appropriate colormap if necessary
    plt.title(f"Compressed Image - Compression Ratio = {compression_ratio:.2f}")  # Set the title with compression ratio
    plt.axis("off")  # Turn off axis labels to focus on the image itself
    plt.savefig(compressed_img_filename)  # Save the image as a .jpeg file with the computed filename
    plt.show()  # Show the image in the plot

##Plot

###Plot Image

In [34]:
def plot_images(
    img: np.ndarray,
    compressed_img: np.ndarray,
    psnr: float,
    compression_ratio: float,
    encoded_img: list[np.ndarray]
    | tuple[
        list[np.ndarray],
        list[np.ndarray],
        list[np.ndarray],
    ],
    color: bool,
) -> None:
    # Create a subplot with 1 row and 2 columns for displaying the images
    fig, axs = plt.subplots(1, 2, figsize=(10, 5))

    # Set the title of the whole figure to display the compression ratio
    fig.suptitle(
        "Compression Ratio = {:.2f}".format(compression_ratio)
    )
    print(f"compression : {compression_ratio}")  # Print the compression ratio for debugging

    # Generate a filename for saving the compressed image based on the compression ratio
    compressed_img_filename = f"compressed_{compression_ratio:.2f}.jpeg"

    # Open a text file to write the encoded image data
    with open("encoded_image.txt", "w") as f:
        if color:
            # If the image is in color, convert from BGR to RGB before displaying
            axs[0].imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))  # Display the original image
            axs[1].imshow(cv.cvtColor(compressed_img, cv.COLOR_BGR2RGB))  # Display the compressed image

            # Write the encoded image data to the text file
            for row in zip(*encoded_img):
                for element in row:
                    f.write(str(element) + " ")
                f.write("\n")
        else:
            # If the image is grayscale, display with a gray colormap
            axs[0].imshow(img, cmap="gray")  # Display the original grayscale image
            axs[1].imshow(compressed_img, cmap="gray")  # Display the compressed grayscale image

            # Write the encoded image data to the text file
            for row in encoded_img:
                for element in row:
                    f.write(str(element) + " ")
                f.write("\n")

    # Uncomment the line below to save the compressed image (currently disabled)
    # save_compressed_image(compressed_img, compression_ratio)

    # Set titles for each subplot (original and compressed images)
    axs[0].set_title("Original Image")
    axs[1].set_title("Compressed Image")

    # Display the images and the compression results
    plt.show()

###Plot Graph

In [35]:
def plot_graph(
    img_dir_path: str,  # Path to the directory containing image files
    color: bool,  # Boolean indicating if the images are in color
):
    psnr_list = []  # List to store the average PSNR values for different coefficients
    compression_ratio_list = []  # List to store the average compression ratios for different coefficients

    # Loop through different numbers of coefficients to analyze their effect on compression
    for num_coefficients in [1, 3, 6, 10, 15, 28]:
        psnr_values = []  # Temporary list to store PSNR values for each image
        compression_ratio_values = []  # Temporary list to store compression ratios for each image

        # Loop through all images in the specified directory
        for img_file in os.listdir(img_dir_path):
            img_path = os.path.join(img_dir_path, img_file)  # Get full path of the image file

            # Analyze the image by performing JPEG compression and retrieving relevant data
            _, _, psnr, compression_ratio, _, _ = analyze_image(
                img_path, 8, num_coefficients, color
            )

            # Append the PSNR and compression ratio for this image to their respective lists
            psnr_values.append(psnr)
            compression_ratio_values.append(compression_ratio)

        # Compute the average PSNR and compression ratio across all images for the current coefficient count
        psnr_list.append(np.mean(psnr_values))
        compression_ratio_list.append(np.mean(compression_ratio_values))

    # Plot the average PSNR vs Compression Ratio on a scatter plot
    plt.plot(compression_ratio_list, psnr_list, "o")

    # Label the x and y axes
    plt.xlabel("Compression Ratio")
    plt.ylabel("PSNR")

    # Set the title of the graph
    plt.title("PSNR vs Compression Ratio")

    # Display the plot
    plt.show()

##Main Function

In [ ]:
if __name__ == "__main__":  # Check if this script is being run directly (not imported as a module)
    """
    Replace the image path with the path to your image
    plot_images function plots the original and compressed images
    Also, it writes the encoded images to a text file encoded_image.txt
    """
    # plot_images(*analyze_image(img_path="path/to/your/image", block_size=8, num_coefficients=10, color=True))

    """
    Replaces the images folder with the path to your images folder
    plot_graph function plots the PSNR vs Compression Ratio graph
    for all the images in the images folder for different values of num_coefficients
    """
    # plot_graph(img_dir_path="path/to/your/image/folder", color=False)

    # Prompt user if they want to analyze a single image
    if input("Analyze a single image (y/n): ") == "y":
        # If user chooses to analyze a single image
        img_path = input("Enter the path to the image: ")  # Ask for the image path
        block_size = int(input("Enter the block size (even): "))  # Ask for the block size (should be even)
        num_coefficients = int(input("Enter the number of coefficients passed: "))  # Ask for number of coefficients
        color = input("Is the image color (y/n): ") == "y"  # Ask if the image is color (convert to boolean)

        # Analyze the image and plot the original and compressed images
        plot_images(*analyze_image(img_path, block_size, num_coefficients, color))

    # Prompt user if they want to analyze all images in a folder
    elif input("Analyzes all images in a folder (y/n): ") == "y":
        # If user chooses to analyze all images in a folder
        img_dir_path = input("Enter the path to the images folder: ")  # Ask for the path to the folder containing images
        color = input("Are the images color (y/n): ") == "y"  # Ask if the images are color (convert to boolean)

        # Generate and display the PSNR vs Compression Ratio graph for all images in the folder
        plot_graph(img_dir_path, color)  # Call plot_graph to analyze all images in the folder